# MMI 702 PROJECT - PART3 : SVM RESULTS

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
from sklearn.preprocessing import LabelEncoder
import glob
import math
from sklearn import svm
import joblib
import numpy as np
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.model_selection import learning_curve
from sklearn.model_selection import ShuffleSplit
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [ ]:
#Read extracted features in part1 notebook
features_1D_mfcc_13 = pd.read_pickle("./features/mfcc_13_features.pk1")
features_1D_mfcc_32 = pd.read_pickle("./features/mfcc_32_features.pk1")
features_1D_only_hnr = pd.read_pickle("./features/hnr_features.pk1")
features_1D_only_zcr = pd.read_pickle("./features/zcr_features.pk1")

#Examples for other features
# features_1D_only_meanf0 = pd.read_pickle("./meanf0_features.pk1")
# features_1D_only_stdevf0 = pd.read_pickle("./stdevf0_features.pk1")
# features_1D_only_sc = pd.read_pickle("./sc_features.pk1")
# features_1D_only_localJitter = pd.read_pickle("./localJitter_features.pk1")
# features_1D_only_localShimmer = pd.read_pickle("./localShimmer_features.pk1")

features_1D_mfcc_copy_0 = pd.read_pickle("./features/mfcc_13_features.pk1")
features_1D_mfcc_copy_1 = pd.read_pickle("./features/mfcc_13_features.pk1")

features_1D_mfcc_32_copy_0 = pd.read_pickle("./features/mfcc_32_features.pk1")
features_1D_mfcc_32_copy_1 = pd.read_pickle("./features/mfcc_32_features.pk1")

In [ ]:
#Convert features to desired form. Convert from pd.DataFrames to numpy arrays. 
# Also merge some features together. Variable names have documentation value.
X_1D_mfcc_13 = np.array(features_1D_mfcc_13.feature.tolist())
y_1D_mfcc_13 = np.array(features_1D_mfcc_13.class_label.tolist())

X_1D_mfcc_32 = np.array(features_1D_mfcc_32.feature.tolist())
y_1D_mfcc_32 = np.array(features_1D_mfcc_32.class_label.tolist())

X_1D_mfcc_cpy_0 = np.array(features_1D_mfcc_copy_0.feature.tolist())
X_1D_hnr_ = np.array(features_1D_only_hnr.feature.tolist())
X_1D_zcr_ = np.array(features_1D_only_zcr.feature.tolist())
X_1D_hnr_ = np.c_[X_1D_mfcc_cpy_0, X_1D_hnr_]
X_1D_hnr_zcr_13 = np.c_[X_1D_hnr_,X_1D_zcr_]
y_1D_hnr_zcr_13 = np.array(features_1D_only_hnr.class_label.tolist())

X_1D_mfcc_32_cpy_0 = np.array(features_1D_mfcc_32_copy_0.feature.tolist())
X_1D_hnr = np.array(features_1D_only_hnr.feature.tolist())
X_1D_zcr = np.array(features_1D_only_zcr.feature.tolist())
X_1D_hnr = np.c_[X_1D_mfcc_32_cpy_0, X_1D_hnr]
X_1D_hnr_zcr_32 = np.c_[X_1D_hnr,X_1D_zcr]
y_1D_hnr_zcr_32 = np.array(features_1D_only_hnr.class_label.tolist())

print("Mfcc_13 shape:", X_1D_mfcc_13.shape)
print("Mfcc_32 shape:", X_1D_mfcc_32.shape)
print("Mfcc_13_hnr_zcr shape:", X_1D_hnr_zcr_13.shape)
print("Mfcc_32_hnr_zcr shape:", X_1D_hnr_zcr_32.shape)
label_encoder = LabelEncoder()

In [ ]:
#Read found best params for SVM in part2 notebook
searchpara_mfcc_13 = joblib.load("./params/params_13_mfcc.pkl")
searchpara_mfcc_32 = joblib.load("./params/params_32_mfcc.pkl")

searchpara_mfcc_13_hnr_zcr = joblib.load("./params/params_13_mfcc_hnr_zcr.pkl")
searchpara_mfcc_32_hnr_zcr = joblib.load("./params/params_32_mfcc_hnr_zcr.pkl")

# searchpara_hnr = joblib.load("params_13_mfcc_hnr.pkl")
# searchpara_meanf0 = joblib.load("params_13_mfcc_meanf0.pkl")
# searchpara_stdevf0 = joblib.load("params_13_mfcc_hnr_stdevf0.pkl")

In [ ]:
#Construct he SVM objects with loaded parameters
SVM_mfcc_13 = svm.SVC(C=searchpara_mfcc_13.best_params_.get('C'), gamma=searchpara_mfcc_13.best_params_.get('gamma'))
SVM_mfcc_32 = svm.SVC(C=searchpara_mfcc_32.best_params_.get('C'), gamma=searchpara_mfcc_32.best_params_.get('gamma'))

SVM_mfcc_13_hnr_zcr = svm.SVC(C=searchpara_mfcc_13_hnr_zcr.best_params_.get('C'), gamma=searchpara_mfcc_13_hnr_zcr.best_params_.get('gamma'))
SVM_mfcc_32_hnr_zcr = svm.SVC(C=searchpara_mfcc_32_hnr_zcr.best_params_.get('C'), gamma=searchpara_mfcc_32_hnr_zcr.best_params_.get('gamma'))

# SVM_hnr = svm.SVC(C=searchpara_hnr.best_params_.get('C'), gamma=searchpara_hnr.best_params_.get('gamma'))
# SVM_meanf0 = svm.SVC(C=searchpara_meanf0.best_params_.get('C'), gamma=searchpara_meanf0.best_params_.get('gamma'))
# SVM_stdevf0 = svm.SVC(C=searchpara_stdevf0.best_params_.get('C'), gamma=searchpara_stdevf0.best_params_.get('gamma'))

In [ ]:
#Function to calculate and display the performance results. 
#To get results, it performs training and testing. 
def plot_learning_curve(
    estimator,
    title,
    X,
    y,
    axes=None,
    ylim=None,
    cv=None,
    n_jobs=None,
    train_sizes=np.linspace(0.1, 1.0, 5),
):
    """
    Generate 3 plots: the test and training learning curve, the training
    samples vs fit times curve, the fit times vs score curve.

    Parameters
    ----------
    estimator : estimator instance
        An estimator instance implementing `fit` and `predict` methods which
        will be cloned for each validation.

    title : str
        Title for the chart.

    X : array-like of shape (n_samples, n_features)
        Training vector, where ``n_samples`` is the number of samples and
        ``n_features`` is the number of features.

    y : array-like of shape (n_samples) or (n_samples, n_features)
        Target relative to ``X`` for classification or regression;
        None for unsupervised learning.

    axes : array-like of shape (3,), default=None
        Axes to use for plotting the curves.

    ylim : tuple of shape (2,), default=None
        Defines minimum and maximum y-values plotted, e.g. (ymin, ymax).

    cv : int, cross-validation generator or an iterable, default=None
        Determines the cross-validation splitting strategy.
        Possible inputs for cv are:

          - None, to use the default 5-fold cross-validation,
          - integer, to specify the number of folds.
          - :term:`CV splitter`,
          - An iterable yielding (train, test) splits as arrays of indices.

        For integer/None inputs, if ``y`` is binary or multiclass,
        :class:`StratifiedKFold` used. If the estimator is not a classifier
        or if ``y`` is neither binary nor multiclass, :class:`KFold` is used.

        Refer :ref:`User Guide <cross_validation>` for the various
        cross-validators that can be used here.

    n_jobs : int or None, default=None
        Number of jobs to run in parallel.
        ``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.
        ``-1`` means using all processors. See :term:`Glossary <n_jobs>`
        for more details.

    train_sizes : array-like of shape (n_ticks,)
        Relative or absolute numbers of training examples that will be used to
        generate the learning curve. If the ``dtype`` is float, it is regarded
        as a fraction of the maximum size of the training set (that is
        determined by the selected validation method), i.e. it has to be within
        (0, 1]. Otherwise it is interpreted as absolute sizes of the training
        sets. Note that for classification the number of samples usually have
        to be big enough to contain at least one sample from each class.
        (default: np.linspace(0.1, 1.0, 5))
    """
    if axes is None:
        _, axes = plt.subplots(1, 3, figsize=(20, 5))

    axes[0].set_title(title)
    if ylim is not None:
        axes[0].set_ylim(*ylim)
    axes[0].set_xlabel("Training examples")
    axes[0].set_ylabel("Score")

    train_sizes, train_scores, test_scores, fit_times, _ = learning_curve(
        estimator,
        X,
        y,
        cv=cv,
        n_jobs=n_jobs,
        train_sizes=train_sizes,
        return_times=True,
    )
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    fit_times_mean = np.mean(fit_times, axis=1)
    fit_times_std = np.std(fit_times, axis=1)

    # Plot learning curve
    axes[0].grid()
    axes[0].fill_between(
        train_sizes,
        train_scores_mean - train_scores_std,
        train_scores_mean + train_scores_std,
        alpha=0.1,
        color="r",
    )
    axes[0].fill_between(
        train_sizes,
        test_scores_mean - test_scores_std,
        test_scores_mean + test_scores_std,
        alpha=0.1,
        color="g",
    )
    axes[0].plot(
        train_sizes, train_scores_mean, "o-", color="r", label="Training score"
    )
    axes[0].plot(
        train_sizes, test_scores_mean, "o-", color="g", label="Cross-validation score"
    )
    axes[0].legend(loc="best")

    # Plot n_samples vs fit_times
    axes[1].grid()
    axes[1].plot(train_sizes, fit_times_mean, "o-")
    axes[1].fill_between(
        train_sizes,
        fit_times_mean - fit_times_std,
        fit_times_mean + fit_times_std,
        alpha=0.1,
    )
    axes[1].set_xlabel("Training examples")
    axes[1].set_ylabel("fit_times")
    axes[1].set_title("Scalability of the model")

    # Plot fit_time vs score
    axes[2].grid()
    axes[2].plot(fit_times_mean, test_scores_mean, "o-")
    axes[2].fill_between(
        fit_times_mean,
        test_scores_mean - test_scores_std,
        test_scores_mean + test_scores_std,
        alpha=0.1,
    )
    axes[2].set_xlabel("fit_times")
    axes[2].set_ylabel("Score")
    axes[2].set_title("Performance of the model")

    return plt




In [ ]:
#Display results
fig, axes = plt.subplots(3, 2, figsize=(10, 15))

title = "Learning Curves - 13_mfcc"
cv = ShuffleSplit(n_splits=100, test_size=0.2, random_state=0)

plot_learning_curve(
    SVM_mfcc_13, title, X_1D_mfcc_13, y_1D_mfcc_13, axes=axes[:, 0], ylim=(0.7, 1.01), cv=cv, n_jobs=4
)

title1 = "Learning Curves - 13_mfcc + hnr + zcr"
cv_1 = ShuffleSplit(n_splits=100, test_size=0.2, random_state=0)

plot_learning_curve(
    SVM_mfcc_13_hnr_zcr, title1, X_1D_hnr_zcr_13, y_1D_hnr_zcr_13, axes=axes[:, 1], ylim=(0.7, 1.01), cv=cv_1, n_jobs=4
)


plt.show()

In [ ]:
#Display results
fig, axes = plt.subplots(3, 2, figsize=(10, 15))

title2 = "Learning Curves - 32_mfcc"
cv_2 = ShuffleSplit(n_splits=100, test_size=0.2, random_state=0)

plot_learning_curve(
    SVM_mfcc_32, title2, X_1D_mfcc_32, y_1D_mfcc_32, axes=axes[:, 0], ylim=(0.7, 1.01), cv=cv, n_jobs=4
)
  

title3 = "Learning Curves - 32_mfcc + hnr + zcr"
cv_3 = ShuffleSplit(n_splits=100, test_size=0.2, random_state=0)

plot_learning_curve(
    SVM_mfcc_32_hnr_zcr, title3, X_1D_hnr_zcr_32, y_1D_hnr_zcr_32, axes=axes[:, 1], ylim=(0.7, 1.01), cv=cv_1, n_jobs=4
)
 
plt.show()